In [5]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import src.config as config

In [7]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1174676
Connected. Call `.close()` to terminate connection gracefully.


In [8]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
    
)
#Use the read_options argument to specify Hive
#df = feature_view.get_batch_data(read_options={"use_hive": True})

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1174676/fs/1165379/fv/time_series_hourly_feature_view/version/1


In [9]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (417.98s) 
2024-11-17 15:36:25,527 WARNING: VersionWarning: Incremented version to `1`.



In [10]:
# drop `pickup_ts` column
ts_data.drop('pickup_ts', axis=1, inplace=True)

# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
1105052,2022-01-01 01:00:00+00:00,0,1
2434999,2022-01-01 02:00:00+00:00,0,1
673857,2022-01-01 03:00:00+00:00,0,1
957084,2022-01-01 04:00:00+00:00,1,1
2136145,2022-01-01 05:00:00+00:00,1,1
...,...,...,...
545672,2024-11-17 05:00:00+00:00,3,265
613418,2024-11-17 06:00:00+00:00,0,265
492630,2024-11-17 07:00:00+00:00,0,265
1032516,2024-11-17 08:00:00+00:00,1,265


In [11]:
# from src.plot import plot_ts
from typing import Optional, List
import pandas as pd
import plotly.express as px 

def plot_ts(
    ts_data: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

plot_ts(ts_data, locations=[43])

In [12]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 265/265 [02:15<00:00,  1.95it/s]

features_and_target.shape=(148003, 675)


In [13]:
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,...,2.0,1.0,2.0,0.0,0.0,2.0,0.0,2022-02-11 16:00:00+00:00,1,1.0
1,0.0,0.0,4.0,1.0,1.0,2.0,1.0,1.0,3.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-13 02:00:00+00:00,1,0.0
2,2.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,3.0,2022-02-14 14:00:00+00:00,1,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2022-02-16 02:00:00+00:00,1,0.0
4,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-17 04:00:00+00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147998,1.0,0.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,4.0,1.0,1.0,3.0,2024-11-13 10:00:00+00:00,215,2.0
147999,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2024-11-14 09:00:00+00:00,215,2.0
148000,1.0,1.0,1.0,0.0,2.0,1.0,2.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2024-11-15 08:00:00+00:00,215,0.0
148001,1.0,4.0,0.0,3.0,0.0,3.0,0.0,0.0,1.0,4.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2024-11-16 07:00:00+00:00,215,3.0


In [15]:
from datetime import date, timedelta
from pytz import timezone
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1), utc=True)

print(f'{cutoff_date=}')

cutoff_date=Timestamp('2024-10-20 00:00:00+0000', tz='UTC')


In [16]:
features_and_target["pickup_hour"] = pd.to_datetime(features_and_target["pickup_hour"])

In [17]:
X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(141729, 674)
y_train.shape=(141729,)
X_test.shape=(6274, 674)
y_test.shape=(6274,)


In [18]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
    
    # sort X_train by `pikup_hour` inplace
    # so the TimeSeriesSplit will split the data in a consistent way
    X_train.sort_values('pickup_hour', inplace=True)

    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [19]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2024-11-17 15:49:03,160] A new study created in memory with name: no-name-5de51894-0e26-418b-83bc-9217a4d3052b


2024-11-17 15:49:03,865 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-11-17 15:49:13,881 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-11-17 15:49:14,894 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

[I 2024-11-17 15:49:29,660] Trial 0 finished with value: 24.035397508302434 and parameters: {'num_leaves': 107, 'feature_fraction': 0.27064983431394707, 'bagging_fraction': 0.6219321099137527, 'min_child_samples': 20}. Best is trial 0 with value: 24.035397508302434.


2024-11-17 15:49:30,108 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-11-17 15:49:51,315 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-11-17 15:49:52,263 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

[I 2024-11-17 15:50:24,093] Trial 1 finished with value: 24.076321066106274 and parameters: {'num_leaves': 109, 'feature_fraction': 0.9786182190564339, 'bagging_fraction': 0.9631344925407164, 'min_child_samples': 98}. Best is trial 0 with value: 24.035397508302434.


...

[I 2024-11-17 15:51:27,856] Trial 2 finished with value: 24.331719407933907 and parameters: {'num_leaves': 205, 'feature_fraction': 0.6050195740117306, 'bagging_fraction': 0.5073000968154059, 'min_child_samples': 79}. Best is trial 0 with value: 24.035397508302434.


2024-11-17 15:51:28,224 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-11-17 15:51:40,559 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-11-17 15:51:41,544 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

[I 2024-11-17 15:52:01,268] Trial 3 finished with value: 24.313017112932542 and parameters: {'num_leaves': 255, 'feature_fraction': 0.27214764840724515, 'bagging_fraction': 0.6221611540226568, 'min_child_samples': 81}. Best is trial 0 with value: 24.035397508302434.


2024-11-17 15:52:01,651 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-11-17 15:52:21,359 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-11-17 15:52:22,509 WARNING: C:\Users\SiliCon\taxi_demand_predictor\src\model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

[I 2024-11-17 15:52:44,885] Trial 4 finished with value: 24.167686255364238 and parameters: {'num_leaves': 145, 'feature_fraction': 0.4493244280137731, 'bagging_fraction': 0.6709926644927964, 'min_child_samples': 57}. Best is trial 0 with value: 24.035397508302434.


In [20]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 107, 'feature_fraction': 0.27064983431394707, 'bagging_fraction': 0.6219321099137527, 'min_child_samples': 20}


In [21]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000021545D311C0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.6219321099137527,
                               feature_fraction=0.27064983431394707,
                               num_leaves=107))])

In [22]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=25.5864


In [23]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['C:\\Users\\SiliCon\\taxi_demand_predictor\\models\\model.pkl']

In [24]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [25]:
model_schema

ModelSchema(input: 'columnar', output: 'columnar')

In [26]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

Connected. Call `.close()` to terminate connection gracefully.


In [27]:
model.save(str(MODELS_DIR / 'model.pkl'))

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/992034 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3402 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/60849 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1174676/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)